In [3]:
from pymongo import MongoClient

# Migrate MySQL to MongoDB
As the database schema has evolved significantly since the current MySQL database was created, we first migrate a subset of the collections using [_MongoDB Relational Migrator_](https://www.mongodb.com/resources/solutions/use-cases/mysql-to-mongodb). We then complete the migration manually using handwritten queries.

In [4]:
client = MongoClient("mongodb://localhost:27017/")
db = client["museo_omero"]

In [5]:
result = db.clienti.find({})

for document in result:
    print(document)

{'_id': ObjectId('68b572f8845fcdbabc1c891f'), 'id': 2, 'nome': 'Mario', 'cognome': 'Orsini', 'tessera': 0, 'cellulare': '+39 (33) 0226 2', 'email': 'heaven.kemmer@gmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8920'), 'id': 6, 'nome': 'Giovanni', 'cognome': 'Casini', 'tessera': 0, 'cellulare': '+39 (33) 5875 7', 'email': 'ureinger@white.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8921'), 'id': 10, 'nome': 'Luigi', 'cognome': 'Re', 'tessera': 0, 'cellulare': '+39 (33) 0474 2', 'email': 'jane63@hotmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8922'), 'id': 16, 'nome': 'Filippo', 'cognome': 'Capriotti', 'tessera': 1, 'cellulare': '+39 (33) 8764 9', 'email': 'roger.murray@gmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8923'), 'id': 25, 'nome': 'Marta', 'cognome': "D'Ambrosio", 'tessera': 1, 'cellulare': '+39 (33) 0122 2', 'email': 'larson.mafalda@gmail.com'}
{'_id': ObjectId('68b572f8845fcdbabc1c8924'), 'id': 28, 'nome': 'Casilde', 'cognome': 'Tambone', 'tessera': 1, 'cellulare': 